# COVID-19 Research Data Analysis Project

This notebook analyzes COVID-19 research papers from the CORD-19 dataset. We'll perform data loading, exploration, cleaning, analysis, and build a Streamlit application.

## Project Overview
- **Part 1**: Data Loading and Basic Exploration
- **Part 2**: Data Cleaning and Preparation
- **Part 3**: Data Analysis and Visualization
- **Part 4**: Streamlit Application
- **Part 5**: Documentation and Reflection

## Section 1: Import Required Libraries

We'll start by importing all the necessary libraries for our analysis.

In [1]:
# Import essential libraries for data analysis and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('default')
sns.set_palette("husl")

print("All libraries imported successfully!")
print("Pandas version:", pd.__version__)
print("NumPy version:", np.__version__)


All libraries imported successfully!
Pandas version: 2.3.3
NumPy version: 2.3.3


## Section 2: Download and Load the Dataset

We'll load the CORD-19 metadata.csv file into a pandas DataFrame. This dataset contains metadata about COVID-19 research papers.

In [ ]:
# Load the CORD-19 metadata dataset
try:
    # Load the CSV file
    df = pd.read_csv('metadata.csv', low_memory=False)
    print(f" Dataset loaded successfully!")
    print(f" Dataset shape: {df.shape[0]} rows, {df.shape[1]} columns")
    print(f" Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
except FileNotFoundError:
    print(" metadata.csv not found. Please ensure the file is in the current directory.")
except Exception as e:
    print(f" Error loading dataset: {e}")

## Section 3: Examine Data Structure and Basic Information

Let's explore the structure of our dataset by examining the first few rows, column names, and data types.

In [ ]:
# Display first few rows
display(df.head())

print("\n Column names:")

for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

print(f"\n Dataset dimensions: {df.shape[0]} rows × {df.shape[1]} columns")

In [ ]:
# Check data types
print("  Data types of each column:")

data_types = df.dtypes
for col, dtype in data_types.items():
    print(f"{col:<25} {dtype}")

print("\n General information:")

df.info()

## Section 4: Data Quality Assessment

Now let's assess the quality of our data by checking for missing values and generating basic statistics.

In [ ]:
# Check for missing values
print(" Missing values analysis:")

missing_data = df.isnull().sum()
missing_percentage = (missing_data / len(df)) * 100

# Create a summary dataframe
missing_summary = pd.DataFrame({
    'Column': missing_data.index,
    'Missing_Count': missing_data.values,
    'Missing_Percentage': missing_percentage.values
})

# Sort by missing percentage
missing_summary = missing_summary.sort_values('Missing_Percentage', ascending=False)

# Display columns with missing values
missing_cols = missing_summary[missing_summary['Missing_Count'] > 0]
print(f" Columns with missing values: {len(missing_cols)} out of {len(df.columns)}")

In [ ]:
# Generate basic statistics for key columns

# Focus on key columns that are likely to be important
key_columns = ['title', 'abstract', 'publish_time', 'journal', 'authors']

for col in key_columns:
    if col in df.columns:
        print(f"\n{col.upper()}:")
        print(f"  Non-null values: {df[col].notna().sum():,}")
        print(f"  Unique values: {df[col].nunique():,}")
        if col == 'publish_time':
            # Try to get date range
            try:
                dates = pd.to_datetime(df[col], errors='coerce')
                print(f"  Date range: {dates.min()} to {dates.max()}")
            except:
                print("  Date conversion failed")
    else:
        print(f"\n{col.upper()}: Column not found")

## Section 5: Handle Missing Data

Based on our analysis, we'll create a cleaned version of the dataset by handling missing values appropriately.

In [ ]:
# Create a cleaned version of the dataset

# Start with a copy of the original data
df_clean = df.copy()

print(f"Original dataset: {df.shape[0]} rows, {df.shape[1]} columns")

# For our analysis, we need papers with titles and some publication info
# Remove rows where title is missing (these are not useful for our analysis)
if 'title' in df_clean.columns:
    df_clean = df_clean.dropna(subset=['title'])
    print(f"After removing rows without titles: {df_clean.shape[0]} rows")

# For publish_time, we'll keep all rows but handle missing dates later
# For other columns, we'll fill missing values with appropriate defaults

# Fill missing journal with 'Unknown'
if 'journal' in df_clean.columns:
    df_clean['journal'] = df_clean['journal'].fillna('Unknown')

# Fill missing abstract with empty string
if 'abstract' in df_clean.columns:
    df_clean['abstract'] = df_clean['abstract'].fillna('')

print(f"Final cleaned dataset: {df_clean.shape[0]} rows, {df_clean.shape[1]} columns")
print(f"Rows removed: {df.shape[0] - df_clean.shape[0]} ({((df.shape[0] - df_clean.shape[0]) / df.shape[0] * 100):.1f}%)")

## Section 6: Data Type Conversions and Feature Engineering

Let's convert date columns and create new features for our analysis.

In [ ]:
# Convert publish_time to datetime and extract year

# Convert publish_time to datetime
if 'publish_time' in df_clean.columns:
    df_clean['publish_date'] = pd.to_datetime(df_clean['publish_time'], errors='coerce')
    
    # Extract year from publication date
    df_clean['publish_year'] = df_clean['publish_date'].dt.year
    
    # For missing dates, we can try to extract year from the string if possible
    missing_dates = df_clean['publish_date'].isna()
    print(f"Rows with missing dates: {missing_dates.sum()}")
    
    # Check the date range
    valid_dates = df_clean['publish_date'].dropna()
    if len(valid_dates) > 0:
        print(f"Date range: {valid_dates.min()} to {valid_dates.max()}")
        print(f"Year range: {df_clean['publish_year'].min()} to {df_clean['publish_year'].max()}")

# Create new features
# Title length
if 'title' in df_clean.columns:
    df_clean['title_length'] = df_clean['title'].str.len()
    print(f"Title length - Mean: {df_clean['title_length'].mean():.1f}, Max: {df_clean['title_length'].max()}")

# Abstract word count
if 'abstract' in df_clean.columns:
    df_clean['abstract_word_count'] = df_clean['abstract'].str.split().str.len()
    # Handle empty abstracts
    df_clean['abstract_word_count'] = df_clean['abstract_word_count'].fillna(0)
    print(f"Abstract word count - Mean: {df_clean['abstract_word_count'].mean():.1f}, Max: {df_clean['abstract_word_count'].max()}")

print(f"\n Feature engineering complete! Dataset now has {df_clean.shape[1]} columns.")

## Section 7: Publication Trends Analysis

Let's analyze publication trends over time to understand how COVID-19 research evolved.

In [ ]:
# Count papers by publication year

if 'publish_year' in df_clean.columns:
    # Count papers by year
    yearly_counts = df_clean['publish_year'].value_counts().sort_index()
    
    print("Papers published by year:")
    for year, count in yearly_counts.items():
        if pd.notna(year):
            print(f"{year}: {count:,} papers")
    
    # Focus on recent years (COVID-19 era)
    covid_years = yearly_counts[yearly_counts.index >= 2019]
    print(f"\nCOVID-19 era (2019+): {covid_years.sum():,} papers")
    
    # Store for visualization
    yearly_data = yearly_counts.reset_index()
    yearly_data.columns = ['Year', 'Paper_Count']
else:
    print("No publication year data available")

# Analyze monthly trends for recent years
if 'publish_date' in df_clean.columns:
    print("\n Monthly publication trends (2020-2022):")
    recent_papers = df_clean[df_clean['publish_year'].between(2020, 2022)]
    if len(recent_papers) > 0:
        monthly_counts = recent_papers.groupby([
            recent_papers['publish_date'].dt.year,
            recent_papers['publish_date'].dt.month
        ]).size().reset_index(name='count')
        
        print(f"Peak months analysis for recent COVID-19 research:")

## Section 8: Journal and Source Analysis

Let's identify the top journals and sources publishing COVID-19 research.

In [ ]:
# Analyze top journals publishing COVID-19 research


if 'journal' in df_clean.columns:
    # Count papers by journal
    journal_counts = df_clean['journal'].value_counts()
    
    print("Top 15 journals by paper count:")
    for i, (journal, count) in enumerate(journal_counts.head(15).items(), 1):
        print(f"{i:2d}. {journal:<50} {count:>6,} papers")
    
    # Store top journals for visualization
    top_journals = journal_counts.head(10).reset_index()
    top_journals.columns = ['Journal', 'Paper_Count']
    
    print(f"\nTotal unique journals: {len(journal_counts)}")
    print(f"Papers in top 10 journals: {top_journals['Paper_Count'].sum():,} ({top_journals['Paper_Count'].sum()/len(df_clean)*100:.1f}%)")

# Check if there are other source columns
potential_source_cols = [col for col in df_clean.columns if 'source' in col.lower()]
if potential_source_cols:
    print(f"\n Other source columns found: {potential_source_cols}")
    
    for col in potential_source_cols[:2]:  # Analyze first 2 source columns
        print(f"\nTop sources in '{col}':")
        source_counts = df_clean[col].value_counts()
        for i, (source, count) in enumerate(source_counts.head(10).items(), 1):
            print(f"{i:2d}. {source:<40} {count:>6,}")
else:
    print("No additional source columns found")

## Section 9: Text Analysis of Paper Titles

Let's analyze the most common words and themes in COVID-19 research paper titles.

In [ ]:
# Analyze word frequency in paper titles

if 'title' in df_clean.columns:
    # Get all titles and convert to lowercase
    all_titles = df_clean['title'].dropna().str.lower()
    
    # Split titles into words and count frequency
    all_words = []
    for title in all_titles:
        # Simple word extraction (remove punctuation and split)
        words = title.replace(',', ' ').replace('.', ' ').replace(':', ' ').replace(';', ' ').replace('(', ' ').replace(')', ' ').split()
        # Filter out very short words and common stop words
        stop_words = {'the', 'and', 'of', 'a', 'an', 'in', 'on', 'at', 'to', 'for', 'with', 'by', 'from', 'as', 'is', 'are', 'was', 'were', 'be', 'been', 'being'}
        words = [word.strip('.,!?:;()[]{}') for word in words if len(word) > 2 and word not in stop_words]
        all_words.extend(words)
    
    # Count word frequencies
    word_counts = Counter(all_words)
    
    print("Top 20 most frequent words in titles:")
    for i, (word, count) in enumerate(word_counts.most_common(20), 1):
        print(f"{i:2d}. {word:<15} {count:>6,} times")
    
    # Store for word cloud
    top_words = dict(word_counts.most_common(50))
    
    # Analyze COVID-related terms
    covid_terms = ['covid', 'coronavirus', 'sars', 'pandemic', 'vaccine', 'vaccination']
    print(f"\nCOVID-19 related terms in titles:")
    for term in covid_terms:
        count = word_counts.get(term, 0)
        if count > 0:
            print(f"  {term}: {count:,} times")
    
    print(f"\nTotal unique words: {len(word_counts)}")
    print(f"Total words analyzed: {len(all_words):,}")
else:
    print("No title data available for analysis")

## Section 10: Create Data Visualizations

Now let's create visualizations to better understand our data patterns.

In [ ]:
# Create publication trends visualization
plt.figure(figsize=(14, 10))

# 1. Publications over time
plt.subplot(2, 2, 1)
if 'yearly_data' in locals():
    # Filter for reasonable years
    yearly_filtered = yearly_data[yearly_data['Year'].between(2000, 2024)]
    plt.plot(yearly_filtered['Year'], yearly_filtered['Paper_Count'], marker='o', linewidth=2, markersize=6)
    plt.title('COVID-19 Research Publications by Year', fontsize=14, fontweight='bold')
    plt.xlabel('Year')
    plt.ylabel('Number of Papers')
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)

# 2. Top journals bar chart
plt.subplot(2, 2, 2)
if 'top_journals' in locals():
    # Get top 8 journals for better readability
    top_8_journals = top_journals.head(8)
    bars = plt.bar(range(len(top_8_journals)), top_8_journals['Paper_Count'], color='skyblue', edgecolor='navy')
    plt.title('Top Journals Publishing COVID-19 Research', fontsize=14, fontweight='bold')
    plt.xlabel('Journals')
    plt.ylabel('Number of Papers')
    plt.xticks(range(len(top_8_journals)), [j[:20] + '...' if len(j) > 20 else j for j in top_8_journals['Journal']], 
               rotation=45, ha='right')
    
    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                f'{int(height):,}', ha='center', va='bottom', fontsize=10)

# 3. Title length distribution
plt.subplot(2, 2, 3)
if 'title_length' in df_clean.columns:
    title_lengths = df_clean['title_length'].dropna()
    plt.hist(title_lengths, bins=50, color='lightgreen', edgecolor='darkgreen', alpha=0.7)
    plt.title('Distribution of Paper Title Lengths', fontsize=14, fontweight='bold')
    plt.xlabel('Title Length (characters)')
    plt.ylabel('Number of Papers')
    plt.axvline(title_lengths.mean(), color='red', linestyle='--', label=f'Mean: {title_lengths.mean():.1f}')
    plt.legend()

# 4. Abstract word count distribution
plt.subplot(2, 2, 4)
if 'abstract_word_count' in df_clean.columns:
    abstract_counts = df_clean['abstract_word_count'].dropna()
    # Filter out extreme outliers for better visualization
    filtered_counts = abstract_counts[abstract_counts <= abstract_counts.quantile(0.95)]
    plt.hist(filtered_counts, bins=50, color='orange', edgecolor='darkorange', alpha=0.7)
    plt.title('Distribution of Abstract Word Counts', fontsize=14, fontweight='bold')
    plt.xlabel('Abstract Word Count')
    plt.ylabel('Number of Papers')
    plt.axvline(filtered_counts.mean(), color='red', linestyle='--', label=f'Mean: {filtered_counts.mean():.1f}')
    plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Create a simple word cloud visualization using matplotlib
plt.figure(figsize=(12, 6))

# Word cloud alternative using bar chart
plt.subplot(1, 2, 1)
if 'top_words' in locals():
    # Get top 15 words for better readability
    top_15_words = dict(list(top_words.items())[:15])
    words = list(top_15_words.keys())
    counts = list(top_15_words.values())
    
    bars = plt.barh(range(len(words)), counts, color='lightcoral', edgecolor='darkred')
    plt.title('Most Frequent Words in Paper Titles', fontsize=14, fontweight='bold')
    plt.xlabel('Frequency')
    plt.ylabel('Words')
    plt.yticks(range(len(words)), words)
    plt.gca().invert_yaxis()  # Most frequent at top
    
    # Add value labels
    for i, bar in enumerate(bars):
        width = bar.get_width()
        plt.text(width + width*0.01, bar.get_y() + bar.get_height()/2,
                f'{int(width):,}', ha='left', va='center', fontsize=10)

# COVID-19 related terms visualization
plt.subplot(1, 2, 2)
if 'word_counts' in locals():
    covid_related = ['covid', 'coronavirus', 'sars', 'pandemic', 'vaccine', 'vaccination', 'treatment', 'patient', 'clinical']
    covid_data = [(word, word_counts.get(word, 0)) for word in covid_related if word_counts.get(word, 0) > 0]
    
    if covid_data:
        covid_words, covid_counts = zip(*covid_data)
        bars = plt.bar(range(len(covid_words)), covid_counts, color='steelblue', edgecolor='navy')
        plt.title('COVID-19 Related Terms in Titles', fontsize=14, fontweight='bold')
        plt.xlabel('Terms')
        plt.ylabel('Frequency')
        plt.xticks(range(len(covid_words)), covid_words, rotation=45, ha='right')
        
        # Add value labels
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                    f'{int(height):,}', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()



In [ ]:
# Save the cleaned data to a new CSV file

# Saving only a sample to avaoid large file sizes
df_clean.sample(frac=0.1).to_csv('covid19_cleaned_sample.csv', index=False)
